In [1]:
import cv2 as ocr
import sys
import gc
import numpy as np
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import math
from tqdm import tqdm
import keras
from keras.utils import np_utils
import pandas as pd
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Input, Conv2D, Dense,MaxPooling2D, Dropout, Flatten
import tensorflow as tf
import keras
import pandas as pd
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
from tqdm import tqdm, trange

Using TensorFlow backend.
C:\Anaconda3\Lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [53]:


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(tf.Session(config=config))
input_shape  = (64,64,1)
def model():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3, 3),activation='sigmoid',input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='sigmoid'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(64, (3, 3), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(128, (3, 3), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(256, (1, 1), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(2, (1, 1), activation='sigmoid'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())


    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
    #               optimizer = keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [49]:
def contrastive_loss(model1, model2, y, margin):
    with tf.name_scope("contrastive-loss"):
        d = tf.sqrt(tf.reduce_sum(tf.pow(model1-model2, 2), 1, keep_dims=True))
        tmp= y * tf.square(d)    
        tmp2 = (1 - y) * tf.square(tf.maximum((margin - d),0))
        return tf.reduce_mean(tmp + tmp2) /2

In [58]:
img = ocr.imread('data/TrainingSet/0007a_num3.png', 0)
img = (255.01 - img)/255
x = img.reshape(-1,64,64,1)
y = 1
y = keras.utils.to_categorical(y)
y = y.reshape(1,2)
print(y)
leftModel = model()
leftModel.summary()
hist = leftModel.fit(x, y, epochs = 1, verbose = 2) 

[[ 0.  1.]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
max_pooling2d_83 (MaxPooling (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 3, 3, 2)           66       

In [ ]:
import tensorflow as tf
dim = 64
l_  = tf.placeholder(tf.float32, [dim,dim,1], name='left')
r_  = tf.placeholder(tf.float32, [dim], name='right')

model.layers[0].output


In [59]:
from keras import backend as K

inp = leftModel.input                                           # input placeholder
outputs = [layer.output for layer in leftModel.layers]          # all layer outputs
functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

# Testing
img1 = ocr.imread('data/TrainingSet/0002a_num3.png', 0)

img1 = (255.01 - img1)/255
# ocr.imshow('image',img1)
# ocr.waitKey(0)
test = img1.reshape(-1,64,64,1)
print(test.shape)
layer_outs = [func([test, 1.]) for func in functors]
im = np.array(layer_outs[1][0].reshape(32,60,60))
im.shape


(1, 64, 64, 1)


(32, 60, 60)

In [60]:
for i in im:
    ocr.imshow('img',i)
    ocr.waitKey(0)

KeyboardInterrupt: 

In [62]:
img1 = ocr.imread('data/TrainingSet/0002a_num3.png', 0)
img1 = (255.01 - img1)/255
x = img1.reshape(-1,64,64,1)
leftModel.predict(img1.reshape(-1,64,64,1))

array([[ 0.59850597,  0.71027213]], dtype=float32)